In [1]:
%matplotlib inline

import os
import datetime

import tqdm
import PIL.Image
import PIL.ExifTags

import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
import subprocess

class VideoEncoder:
    
    def __init__(self, ffmpeg_path='ffmpeg'):
        self.ffmpeg = ffmpeg_path
        self.proc = None
        
    def start(self, outfile, framerate=1, vcodec='mjpeg'):
        if self.proc:
            raise RuntimeError('Calling "start()" before process pipe was closed.')

        cmd = (self.ffmpeg,
                     '-y',
                     '-r', str(framerate),
                     '-f','image2pipe',
                     '-vcodec', vcodec,
                     '-i', 'pipe:', 
                     '-vcodec', 'libxvid', outfile)
        
        self.proc = subprocess.Popen(cmd, stdin=subprocess.PIPE)
        
    def encode(self, framelist):
        ''' `framelist` must be an iterable of raw bytes '''
        for frame in framelist:
            self.proc.stdin.write(frame)
            
    def close(self):
        self.proc.stdin.close()
        self.proc = None

In [3]:
# Build a map of filename -> timestamp, camera, resolution
TS_FORMAT = '%Y:%m:%d %H:%M:%S'
ts_map = {}
for fname in [f for f in os.listdir('session') if os.path.isfile(os.path.join('session', f))]:
    img = PIL.Image.open(os.path.join('session', fname))
    exif = {
        PIL.ExifTags.TAGS[k]: v
        for k, v in img._getexif().items()
        if k in PIL.ExifTags.TAGS
    }
    timestamp = datetime.datetime.strptime(exif.get('DateTime'), TS_FORMAT)
    ts_map[fname] = (timestamp, exif.get('Model'), '%dx%d' % (exif.get('ExifImageHeight'), exif.get('ExifImageWidth')))

In [4]:
# Compute the difference between frames
df = pd.DataFrame.from_dict(ts_map, orient='index').reset_index()\
    .rename(columns={'index': 'filename', 0: 'timestamp', 1: 'camera', 2: 'resolution'}).sort_values('timestamp')
df['timediff'] = df['timestamp'].diff().astype('timedelta64[s]')
df.head()

,filename,timestamp,camera,resolution,timediff
137,G0010205.JPG,2016-07-26 13:06:13,HERO4 Session          ,2448x3264,NaN
2866,G0020206.JPG,2016-07-26 13:07:35,HERO4 Session          ,2448x3264,82.0
2097,G0030207.JPG,2016-07-26 13:16:26,HERO4 Session          ,2448x3264,531.0
2633,GOPR0209.JPG,2016-07-26 13:49:54,HERO4 Session          ,2040x2720,2008.0
3304,GOPR0210.JPG,2016-07-26 13:50:23,HERO4 Session          ,2040x2720,29.0


In [5]:
# Assign values to contiguous frames
tmp = df['timediff'] > 120
df['framegroup'] = tmp.astype(int).cumsum()
df.head()

,filename,timestamp,camera,resolution,timediff,framegroup
137,G0010205.JPG,2016-07-26 13:06:13,HERO4 Session          ,2448x3264,NaN,0
2866,G0020206.JPG,2016-07-26 13:07:35,HERO4 Session          ,2448x3264,82.0,0
2097,G0030207.JPG,2016-07-26 13:16:26,HERO4 Session          ,2448x3264,531.0,1
2633,GOPR0209.JPG,2016-07-26 13:49:54,HERO4 Session          ,2040x2720,2008.0,2
3304,GOPR0210.JPG,2016-07-26 13:50:23,HERO4 Session          ,2040x2720,29.0,2


In [6]:
# Filter out frames with bogus resolution
df['filter'] = 1
df['area'] = df['resolution'].apply(lambda x: int(x.split('x')[0]) * int(x.split('x')[1]))
for group in df['framegroup'].unique():
    ix = df['framegroup'] == group
    median = df.loc[ix, 'area'].median()
    df.loc[ix, 'filter'] = (df.loc[ix, 'area'] == median).astype(int)

In [9]:
for group in df['framegroup'].unique():
    ix = (df['framegroup'] == group) & df['filter']
    frames = df.loc[ix]
    if len(frames) < 10: continue
        
    mintime = frames['timestamp'].min()
    maxtime = frames['timestamp'].max()
    
    avediff = frames['timediff'].mean()
    framerate = 8.0 * 60.0 / avediff

    print('Time span: %s - %s (%s)' % (mintime, maxtime, (maxtime - mintime)))
    print('Framerate: %f (~%f)' % (framerate, avediff))
    print()

Time span: 2016-07-26 16:54:46 - 2016-07-27 11:15:55 (0 days 18:21:09)
Framerate: 14.984908 (~32.032229)

Time span: 2016-07-27 16:26:13 - 2016-07-28 10:45:43 (0 days 18:19:30)
Framerate: 7.997458 (~60.019074)



In [10]:
enc = VideoEncoder()
for group in df['framegroup'].unique():
    frames = df[df['filter'] & (df['framegroup'] == group)]
    if len(frames) < 10: continue
    framerate = 8.0 * 60.0 / frames['timediff'].mean()
    enc.start('seastars_%02d.mp4' % group, framerate=framerate)
    for fname in tqdm.tqdm(frames['filename'][:]):
        img = open(os.path.join('session', fname), 'rb')
        enc.encode([img.read()])
    enc.close()

100%|██████████████████████████████████████████████████████████████████████████████| 1101/1101 [07:50<00:00,  2.29it/s]
